In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pysam
from collections import defaultdict

# === USER INPUT ===
clean_file = "path/to/clean.sam"
contaminated_file = "path/to/contaminated.sam"
bin_size = 1_000_000  # bin size in base pairs
output_plot = "mapq_over_position.png"

# === FUNCTION TO PARSE AND BIN MAPQ BY POSITION ===
def get_binned_mapq(sam_path, bin_size):
    binned_mapq = defaultdict(list)
    with pysam.AlignmentFile(sam_path, "r") as samfile:
        for read in samfile.fetch(until_eof=True):
            if not read.is_unmapped:
                chrom = read.reference_name
                pos = read.reference_start
                bin_id = (chrom, pos // bin_size)
                binned_mapq[bin_id].append(read.mapping_quality)
    return binned_mapq

# === FUNCTION TO AVERAGE MAPQ PER BIN ===
def average_binned_mapq(binned_mapq):
    averaged = {}
    for bin_id, values in binned_mapq.items():
        averaged[bin_id] = np.mean(values)
    return averaged

# === PROCESS FILES ===
cont_binned = get_binned_mapq(contaminated_file, bin_size)
clean_binned = get_binned_mapq(clean_file, bin_size)

cont_avg = average_binned_mapq(cont_binned)
clean_avg = average_binned_mapq(clean_binned)

# === MERGE AND SORT BIN POSITIONS ===
all_bins = sorted(set(cont_avg.keys()).union(clean_avg.keys()), key=lambda x: (x[0], x[1]))
x_labels = [f"{chrom}:{bin_id*bin_size//1_000_000}Mb" for chrom, bin_id in all_bins]
x_indices = np.arange(len(all_bins))

# === EXTRACT PLOT DATA ===
cont_values = [cont_avg.get(bin_id, np.nan) for bin_id in all_bins]
clean_values = [clean_avg.get(bin_id, np.nan) for bin_id in all_bins]

# === PLOT ===
plt.figure(figsize=(15, 6))
plt.plot(x_indices, cont_values, label="Contaminated", color="orange")
plt.plot(x_indices, clean_values, label="Clean", color="blue")
plt.xticks(x_indices[::max(len(x_indices)//10,1)], x_labels[::max(len(x_labels)//10,1)], rotation=45)
plt.ylabel("Mean MAPQ per 1Mb bin")
plt.xlabel("Genomic Position (binned)")
plt.title("Mapping Quality over Genomic Position")
plt.legend()
plt.tight_layout()
plt.savefig(output_plot, dpi=300)
plt.close()

print(f"Plot saved as {output_plot}")